# DISEASE OUTBREAK PREDICTION

importing necessary libraries

In [ ]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd # parallel processing for pandas (to read large datasets) fixed memory issue
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from datetime import datetime, timezone
nltk.download('wordnet')

Reading Datasets

In [2]:
# reading comments dataset (do not have enough computational power and memory space to read the entire dataset)
data_comments = pd.read_csv('C:\\Users\\kedar\\OneDrive\\Desktop\\ML mini project\\DiseaseOutbreak_Dataset\\the-reddit-covid-dataset-comments.csv', nrows=1000000)

In [3]:
#checking whether the data loaded
data_comments.head()

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score
0,comment,hi1vsag,2riyy,nova,False,1635206399,https://old.reddit.com/r/nova/comments/qfs53d/...,When you scheduled your booster with CVS does ...,0.0000,2
1,comment,hi1vs7i,2qhov,vancouver,False,1635206397,https://old.reddit.com/r/vancouver/comments/qf...,Didn't stop prices there though. New Zealand a...,0.1887,32
2,comment,hi1vs5n,2qwzb,pregnant,False,1635206397,https://old.reddit.com/r/pregnant/comments/qfs...,I’m just waiting until after pregnancy to get ...,0.6720,1
3,comment,hi1vs5v,2qixm,startrek,False,1635206397,https://old.reddit.com/r/startrek/comments/qft...,*The first duty of every Starfleet officer is ...,0.9562,1
4,comment,hi1vs0l,2qsf3,ontario,False,1635206395,https://old.reddit.com/r/ontario/comments/qfkj...,Compare BC to Ontario for COVID. It's even mor...,0.0000,-2


In [4]:
# reading posts dataset
data_posts = pd.read_csv('C:\\Users\\kedar\\OneDrive\\Desktop\\ML mini project\\DiseaseOutbreak_Dataset\\the-reddit-covid-dataset-posts.csv',nrows=1000000)

In [5]:
# checking whetehr the dataset is loaded
data_posts.head()

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
0,post,qftvfn,3h9d4,autonewspaper,False,1635206379,https://old.reddit.com/r/AutoNewspaper/comment...,washingtontimes.com,https://www.washingtontimes.com/news/2021/oct/...,NaN,"[Politics] - Kay Ivey, Alabama governor, direc...",1
1,post,qftusg,2x4yx,coronavirus,False,1635206322,https://old.reddit.com/r/Coronavirus/comments/...,twitter.com,NaN,[deleted],More Floridians dead from COVID than Vietnam W...,1
2,post,qftugv,3h9d4,autonewspaper,False,1635206292,https://old.reddit.com/r/AutoNewspaper/comment...,nzherald.co.nz,https://www.nzherald.co.nz/nz/news/article.cfm...,NaN,[NZ] - Covid-19 coronavirus Delta outbreak: La...,1
3,post,qftuge,3h9d4,autonewspaper,False,1635206291,https://old.reddit.com/r/AutoNewspaper/comment...,nzherald.co.nz,https://www.nzherald.co.nz/nz/news/article.cfm...,NaN,[NZ] - Covid 19 Delta outbreak: Man arrested a...,1
4,post,qftucd,4egz78,cointuta,False,1635206281,https://old.reddit.com/r/CoinTuta/comments/qft...,cointuta.com,https://www.cointuta.com/covid-19-more-likely-...,NaN,Covid-19 more likely to cause neurological iss...,1


Data Preprocessing

In [8]:
# analysing missing values
nan_comments = data_comments.isna().sum()
nan_posts = data_posts.isna().sum()
print('Nan values in column of Comments dataset: ',nan_comments)
print('Nan values in each column of Posts dataset: ',nan_posts)

Nan values in column of Comments dataset:  type                  0
id                    0
subreddit.id          0
subreddit.name        0
subreddit.nsfw        0
created_utc           0
permalink             0
body                  0
sentiment         37937
score                 0
dtype: int64
Nan values in each column of Posts dataset:  type                   0
id                     0
subreddit.id           0
subreddit.name         0
subreddit.nsfw         0
created_utc            0
permalink              0
domain                 4
url               257459
selftext          742541
title                  0
score                  0
dtype: int64


In [9]:
# lots of missing values in comments dataset - sentiment column
# lots of missing values in posts dataset - url and selftext columns
# dropping these columns
data_comments = data_comments.drop(columns = 'sentiment')
data_posts = data_posts.drop(columns = ['url','selftext'])

In [10]:
# checking if chenges were made
print('Columns in comments dataset: ',data_comments.columns)
print('Columns in posts dataset: ',data_posts.columns)

# since domain has missing values filling nan values with most frequently occuring value
data_posts['domain'].fillna(data_posts['domain'].mode()[0], inplace=True)

# re-analysing missing values
nan_comments = data_comments.isna().sum()
nan_posts = data_posts.isna().sum()
print('Nan values in column of Comments dataset: ',nan_comments)
print('Nan values in each column of Posts dataset: ',nan_posts)

Columns in comments dataset:  Index(['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw',
       'created_utc', 'permalink', 'body', 'score'],
      dtype='object')
Columns in posts dataset:  Index(['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw',
       'created_utc', 'permalink', 'domain', 'title', 'score'],
      dtype='object')
Nan values in column of Comments dataset:  type              0
id                0
subreddit.id      0
subreddit.name    0
subreddit.nsfw    0
created_utc       0
permalink         0
body              0
score             0
dtype: int64
Nan values in each column of Posts dataset:  type              0
id                0
subreddit.id      0
subreddit.name    0
subreddit.nsfw    0
created_utc       0
permalink         0
domain            0
title             0
score             0
dtype: int64


In [11]:
# converting all utc objects to date-time objects
data_comments['created_utc'] = pd.to_datetime(data_comments['created_utc'], unit='s', utc=True)
data_posts['created_utc'] = pd.to_datetime(data_posts['created_utc'], unit='s', utc=True)

In [12]:
# checking 
print('datatype of creted utc column in comments dataset: ',data_comments['created_utc'].dtype)
print('datatype of creted utc column in posts dataset: ',data_posts['created_utc'].dtype)

datatype of creted utc column in comments dataset:  datetime64[ns, UTC]
datatype of creted utc column in posts dataset:  datetime64[ns, UTC]


In [21]:
# checking for duplicate values in the column 'subreddit.id' since no two subreddits can have the same values

# Check for duplicated values in 'subreddit.id' and 'body'
duplicates_comments = data_comments[data_comments.duplicated(subset=['subreddit.id', 'body'], keep=False)]

# Keeping only the first occurrence of each unique combination of 'subreddit.id' and 'body'
data_comments = data_comments[duplicates_comments != True]

In [23]:
# checking for duplicate values in the column 'subreddit.id' since no two subreddits can have the same values
# Check for duplicated values in 'subreddit.id' and 'body'
duplicates_posts = data_posts[data_posts.duplicated(subset=['subreddit.id', 'title'], keep=False)]

# Keeping only the first occurrence of each unique combination of 'subreddit.id' and 'body'
data_posts = data_posts[duplicates_posts != True]

In [ ]:
https://abhishek-maheshwarappa.medium.com/installing-cuda-for-nvidia-gpu-and-cudnn-on-windows-10-f03e6e82f265